Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.605173
Minibatch accuracy: 18.8%
Validation accuracy: 8.0%
Minibatch loss at step 50: 1.415088
Minibatch accuracy: 62.5%
Validation accuracy: 59.6%
Minibatch loss at step 100: 0.882063
Minibatch accuracy: 75.0%
Validation accuracy: 71.2%
Minibatch loss at step 150: 0.710479
Minibatch accuracy: 75.0%
Validation accuracy: 72.0%
Minibatch loss at step 200: 0.651707
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 250: 0.777817
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.379353
Minibatch accuracy: 93.8%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.472992
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 400: 0.715945
Minibatch accuracy: 68.8%
Validation accuracy: 77.7%
Minibatch loss at step 450: 0.854764
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 500: 1.068245
Minibatch accuracy: 68.8%
Validation accuracy: 81.9%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    h_max = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    conv = tf.nn.conv2d(h_max, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    h_max = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    shape = h_max.get_shape().as_list()
    reshape = tf.reshape(h_max, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.962821
Minibatch accuracy: 6.2%
Validation accuracy: 8.2%
Minibatch loss at step 50: 1.818185
Minibatch accuracy: 31.2%
Validation accuracy: 38.3%
Minibatch loss at step 100: 0.797913
Minibatch accuracy: 75.0%
Validation accuracy: 63.8%
Minibatch loss at step 150: 0.739520
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%
Minibatch loss at step 200: 0.452556
Minibatch accuracy: 87.5%
Validation accuracy: 77.5%
Minibatch loss at step 250: 0.647848
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 300: 0.224456
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.209054
Minibatch accuracy: 93.8%
Validation accuracy: 75.9%
Minibatch loss at step 400: 0.588746
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 450: 1.065001
Minibatch accuracy: 68.8%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.993732
Minibatch accuracy: 62.5%
Validation accuracy: 83.7%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    h_max = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    conv = tf.nn.conv2d(h_max, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    h_max = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
    shape = h_max.get_shape().as_list()
    reshape = tf.reshape(h_max, [shape[0], shape[1] * shape[2] * shape[3]])
    drop =  tf.nn.dropout(reshape, 0.5)
    hidden = tf.nn.relu(tf.matmul(drop, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
#   optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 100000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.169368
Minibatch accuracy: 6.2%
Validation accuracy: 9.4%
Minibatch loss at step 50: 2.177151
Minibatch accuracy: 12.5%
Validation accuracy: 26.7%
Minibatch loss at step 100: 1.684902
Minibatch accuracy: 37.5%
Validation accuracy: 47.7%
Minibatch loss at step 150: 1.261149
Minibatch accuracy: 68.8%
Validation accuracy: 60.0%
Minibatch loss at step 200: 1.032085
Minibatch accuracy: 81.2%
Validation accuracy: 64.7%
Minibatch loss at step 250: 1.339189
Minibatch accuracy: 68.8%
Validation accuracy: 70.2%
Minibatch loss at step 300: 0.668253
Minibatch accuracy: 81.2%
Validation accuracy: 71.6%
Minibatch loss at step 350: 0.432694
Minibatch accuracy: 87.5%
Validation accuracy: 70.8%
Minibatch loss at step 400: 1.041104
Minibatch accuracy: 68.8%
Validation accuracy: 74.3%
Minibatch loss at step 450: 0.998882
Minibatch accuracy: 62.5%
Validation accuracy: 72.8%
Minibatch loss at step 500: 1.696572
Minibatch accuracy: 56.2%
Validation accuracy: 77.5%
Min

Validation accuracy: 85.0%
Minibatch loss at step 4550: 0.503047
Minibatch accuracy: 87.5%
Validation accuracy: 84.9%
Minibatch loss at step 4600: 0.112428
Minibatch accuracy: 93.8%
Validation accuracy: 85.1%
Minibatch loss at step 4650: 0.731095
Minibatch accuracy: 75.0%
Validation accuracy: 85.2%
Minibatch loss at step 4700: 0.114486
Minibatch accuracy: 100.0%
Validation accuracy: 85.4%
Minibatch loss at step 4750: 0.387758
Minibatch accuracy: 93.8%
Validation accuracy: 85.4%
Minibatch loss at step 4800: 0.813782
Minibatch accuracy: 75.0%
Validation accuracy: 85.5%
Minibatch loss at step 4850: 0.582217
Minibatch accuracy: 75.0%
Validation accuracy: 85.3%
Minibatch loss at step 4900: 0.122730
Minibatch accuracy: 93.8%
Validation accuracy: 85.3%
Minibatch loss at step 4950: 0.287652
Minibatch accuracy: 93.8%
Validation accuracy: 85.6%
Minibatch loss at step 5000: 1.056890
Minibatch accuracy: 68.8%
Validation accuracy: 85.1%
Minibatch loss at step 5050: 0.257371
Minibatch accuracy: 87.5

Validation accuracy: 86.5%
Minibatch loss at step 9050: 0.682548
Minibatch accuracy: 81.2%
Validation accuracy: 86.5%
Minibatch loss at step 9100: 0.142125
Minibatch accuracy: 100.0%
Validation accuracy: 86.6%
Minibatch loss at step 9150: 0.500867
Minibatch accuracy: 81.2%
Validation accuracy: 86.5%
Minibatch loss at step 9200: 1.042291
Minibatch accuracy: 81.2%
Validation accuracy: 86.4%
Minibatch loss at step 9250: 0.525992
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 9300: 0.775019
Minibatch accuracy: 75.0%
Validation accuracy: 86.1%
Minibatch loss at step 9350: 0.115071
Minibatch accuracy: 100.0%
Validation accuracy: 86.5%
Minibatch loss at step 9400: 0.248245
Minibatch accuracy: 93.8%
Validation accuracy: 86.5%
Minibatch loss at step 9450: 0.110133
Minibatch accuracy: 93.8%
Validation accuracy: 86.6%
Minibatch loss at step 9500: 0.606237
Minibatch accuracy: 81.2%
Validation accuracy: 86.6%
Minibatch loss at step 9550: 0.257883
Minibatch accuracy: 87.